# Tutorial 5: Where to go from here

In our fourth tutorial, we used a Sagemaker training job to create a model on the full data set.
We learned that SageMaker training jobs allow us to
* monitor and log the training progress,
* keep track of the trainings we ran so far,
* decouple the actual training process from the notebook that prepares and controls our experimental setup.

Our final tutorial is about _model performance_ in terms of _prediction quality_.\
To succeed in our competition, we obviously want to obtain a model that maximizes prediction quality.
But how do we get there?

To answer this question, we will
1. sketch how to approach a data science/machine learning problem in an iterative, hypothesis-driven manner,
2. show how to obtain insights for optimizing our model and machine learning (ML) pipeline by debugging and interpreting our model's predictions, configuration, and input data.

**NOTE: We will NOT need a GPU for this tutorial notebook. Pick a non GPU instance type to save costs.**

In [13]:
import boto3           # For interacting with S3
import pandas as pd
import sys             # Python system library needed to load custom functions

# Imports to run Sagemaker training jobs
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from sagemaker.session import Session

In [14]:
sys.path.append('../src')  # Add the source directory to the PYTHONPATH. This allows to import local functions and modules.

In [ ]:
from config import DEFAULT_BUCKET, DEFAULT_REGION  # The name of the S3 bucket that contains the training data
from detection_util import create_predictions
from gdsc_util import download_and_extract_model, set_up_logging, extract_hyperparams, PROJECT_DIR
from tutorial_4_training import load_config as load_config_4
from tutorial_5_training import load_config as load_config_5
from tutorial_5_training_8_epochs import load_config as load_config_5_with_8_epochs
from gdsc_util import load_sections_df
from PredictionEvaluator import PredictionEvaluator
from gdsc_score import get_leaderboard_score

set_up_logging()  # Sets up logging to console and .log

## Iterative, Hypothesis-driven Model Development

When we start working on a typical data science/ML problem, many aspects of the problem itself, the underlying data, and the required solution are vague.\
Repeated experimentation is key to develop an adequate solution in a systematic manner.\
Therefore, it is best practice to tackle a data science/ML problem in multiple iterations.

The [CRISP-DM process model](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining) defines the relevant phases of each iteration: 

<div style="text-align: left">
    <img src="figures/crisp_dm.png" alt="CRISP-DM" style="width:40%;" />
</div>

Applying this iterative, feedback-driven process allows us to
- better understand the business problem that we want to solve,
- collect, understand, and prepare required data,
- identify and engineer relevant features from our data,
- develop/select and tune a suitable model of adequate complexity.

### The Scientific Method

A good way to plan and to carry out these iterations is to apply the [scientific method](https://en.wikipedia.org/wiki/Scientific_method).\
A key concept of this approach are _hypotheses_:
- A hypothesis is a testable statement that can either be true or false. It formulates a belief how a specific measure will influence our model's performance.
- Hypotheses are tested by conducting an experiment.

__Example:__ "Extending input data for training and inference by different stainings will increase prediction accuracy."

Before the first iteration, we create a list of initial hypotheses and sort it by priority. Priorities can be assigned by expected information gain or risk, for instance.\
Having created such an initial list, each iteration looks as follows:
1. Select the hypothesis of highest priority.
2. Conduct an experiment to test the influence of the measure on the model's performance.
3. Investigate the outcome and use the gained insights to derive further hypotheses. Adjust priorities if required. 

Note that this approach not only helps us in prioritizing our investigations, but also forces us to justify our design decisions.\
As a consequence, different aspects of the problem are better understood and complexity is only added if needed.\
The latter characteristic is especially important when having to adapt to changes in the future.

### A First List of Hypotheses

With regard to our detection problem, we start testing the following hypotheses in the given order:
1. Tuning the MMDetection configuration with respect to the input data will increase prediction quality.
2. Ignoring the model's detection score deteriorates prediction quality.
3. Overfitting impairs our model's generalizability.

In the following sections, we will test these hypotheses to improve our model and guide further investigations.

## Analyzing Model Performance and Tuning the Configuration

Hypothesis:
> Tuning the MMDetection configuration with respect to the input data will increase prediction quality.

To test this hypothesis, let us first take a look at the logs of a training job of Tutorial 4.\
To this end, we navigate to a corresponding SageMaker traning job in the AWS management console and open its log output in CloudWatch (see Tutorial 4).

The log shows that, after 4 epochs, we get a recall of about 0.7. This means that 30% of all worm sections are not detected by our model. The score we got assigned on the [GDSC website](https://gdsc.ce.capgemini.com/) was around 75.\
We should be able to do much better than this!
It seems that our model is currently not able to extract as much information from the data as needed to provide adequate predictions.

Let us check again the configuration we used in Tutorial 4:

In [5]:
data_folder = str(PROJECT_DIR / 'data')
cfg, base_file = load_config_4(data_folder)

print(f'Config:\n{cfg.pretty_text}')

ContentTooShortError: <urlopen error retrieval incomplete: got only 4960 out of 177867103 bytes>

A very careful look at the configuration (or the MMdetection [tutorial](https://mmdetection.readthedocs.io/en/latest/tutorials/config.html)) reveals the interesting parameter ```cfg.model.test_cfg.rcnn.max_per_img```. It limits the maximal number of sections that can be predicted per image.

Currently, this value is set to 100:

In [ ]:
cfg.model.test_cfg.rcnn.max_per_img

Intuitively, we should ensure that the parameter value is not smaller than the maximum of the number of bounding boxes per image.\
So how many bounding boxes do we have per image? Let us extend our data analysis we started in Tutorial 2:

In [ ]:
section_df = load_sections_df(f'{PROJECT_DIR}/data/gdsc_train.csv')
nodules = section_df.groupby(['file_name','staining','study']).size().sort_values(ascending=False).reset_index()  # Group the data by nodule image
nodules[0].hist()

In [ ]:
display(nodules.head(5))
fraction = len(nodules[nodules[0] > 100])/float(len(nodules))
print('Fraction >100: {:.1%}'.format(fraction))

According to the histogram, there are quite many nodules with 100-200 sections.
The table above shows that we even have 4 nodules with more than 300 sections. So way more than our current limit of 100.
In fact, 19% of all nodules in our training set feature more than 100 sections.

So it is valid to assume that the current value of ```cfg.model.test_cfg.rcnn.max_per_img``` is too low.
To have some buffer for the test data, we will set it to 400.

We have prepared this adjusted configuration in the training code for Tutorial 5.
So we just load it in the following step.

In [ ]:
data_folder = str(PROJECT_DIR / 'data')
cfg, base_file = load_config_5(data_folder)

print(f'max_per_img: {cfg.model.test_cfg.rcnn.max_per_img}')

We re-train our model to test if the prediction quality of our model increases.

Note that we also extended our training script by two prediction steps on train and test data.
These predictions are created automatically after the training is completed. They are stored in two CSV files &mdash; one for "train" and another one for "test". After downloading and extracting the generated model, both files are located in your project's data folder. This is the same location where you found the generated predictions of Tutorial 4.

This functionality will become handy in the course of this tutorial.

In [ ]:
entry_point = 'tutorial_5_training.py'
exp_name = entry_point.split('.')[0].replace('_', '-')  # AWS does not allow . and _ as experiment names

account_id = boto3.client('sts').get_caller_identity().get('Account')
role = get_execution_role()

sm_client = boto3.client("sagemaker", region_name=DEFAULT_REGION)
sess = Session(sagemaker_client=sm_client)
s3_output_location = f"s3://{sess.default_bucket()}/{exp_name}"
input_channels = {"train": f"s3://{DEFAULT_BUCKET}"}

hyperparameters = extract_hyperparams(entry_point) # custom function to parse the training script and extract config
hyperparameters['base_file'] = base_file

metrics = [
    {"Name": "train:loss_rpn_cls", "Regex": "loss_rpn_cls: ([0-9\.]+)"},
    {"Name": "train:loss_rpn_bbox", "Regex": "loss_rpn_bbox: ([0-9\.]+)"},
    {"Name": "train:loss_cls", "Regex": "loss_cls: ([0-9\.]+)"},
    {"Name": "train:loss_bbox", "Regex": "loss_bbox: ([0-9\.]+)"},
    {"Name": "train:loss", "Regex": "loss: ([0-9\.]+)"},
    {"Name": "train:accuracy", "Regex": "acc: ([0-9\.]+)"},
    {"Name": "train:epoch", "Regex": "Epoch (\[[0-9\.]+\])"},
    {"Name": "val:epoch", "Regex": "Epoch\(val\) (\[[0-9]+\])"},
    {"Name": "val:mAP", "Regex": "mAP: ([0-9\.]+)"},
]

estimator = PyTorch(
    entry_point=entry_point,             # This function will be called by the training job
    source_dir="../src",                 # All code in this folder will be copied over
    image_uri=f"954362353459.dkr.ecr.{DEFAULT_REGION}.amazonaws.com/sm-training-custom:torch-1.8.1-cu111-noGPL",
    role=role,
    output_path=s3_output_location,
    container_log_level=20,             # 10=debug, 20=info
    base_job_name=exp_name,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",     # a GPU instance
    volume_size=45,
    metric_definitions=metrics,
    hyperparameters=hyperparameters,
)

In [ ]:
estimator.fit(
    input_channels,
    wait=False,           # Whether or not the notebook should wait for the job to finish. By setting it to False we can continue working while the job runs on another machine.
)

# save the name of the experiment to the filesystem so that we can use it later
experiment_name = estimator._hyperparameters["sagemaker_job_name"]

with open(f'{PROJECT_DIR}/experiment_tut5_max_per_img.txt', 'w+') as f:
    f.write(experiment_name)

In [59]:
# read the experiment name from the filesystem
with open(f'{PROJECT_DIR}/experiment_tut5_max_per_img.txt', 'r') as f:
    experiment_name = f.read()
    

model_location = f'{s3_output_location}/{experiment_name}/output/model.tar.gz'
local_model_dir = download_and_extract_model(model_uri=model_location, local_dir='data')

2022-06-07 17:34:09,897 - gdsc_util - INFO - File tutorial-5-training/tutorial-5-training-2022-05-29-10-30-56-102/output/model.tar.gz already exists. Skipping download


With the parameter adjustment, you should now get a recall of about 0.8 (note that the previous recall was ~0.7).\
Submitting the results on the [GDSC website](https://gdsc.ce.capgemini.com/) reveals a more disappointing result:
_Our score decreased from 75 to 66!_

To understand this behavior, recap that the assigned score is the sum of multiple accuracy values our model obtains for different IoU thresholds (see Tutorial 3, for instance).
Interestingly, a major difference between the accuracy and the recall metrics is that recall does not take account of _false positives_. 

So it is valid to assume that the drop in overall performance is due to an increased number of false positives.
The following observations supprt this assumption:
1. The statistics of the section counts shows that 75% of our nodules do not have more than 86 sections. This is far less than the 400 regions the model can detect. 
2. Checking the training logs reveals that the model detects ~105k regions with the new configuration, compared to ~95k with the old configuration. Many of these additional regions could be false positives given that most nodules exhibit relatively few sections.

In [60]:
nodules.describe()

,0
count,994.000000
mean,66.083501
std,54.383185
min,1.000000
25%,28.250000
50%,53.000000
75%,86.000000
max,381.000000


So far, our hypothesis did not turn out to be _true_.
We still keep the updated configuration since the experiment's outcome supports our idea of investigating the model's detection score in the next section.

## Avoiding False Detections/Positives

Refined hypothesis:
> Ignoring the model's detection score deteriorates prediction quality due to false positives.

Currently, we consider every bounding box that our model draws to be a detected section &mdash; no matter how _certain_ the model is about it.\
In fact, our model also outputs a detection score for every bounding box. It denotes the model's confidence that it actually found a section.
So far, we did not make use of this information.

In the following experiment, we will test if we can improve our model's prediction quality by only submitting sections having high detection scores.

To recap how the predictions look like, we load the predictions on training data created by the model trained using the updated configuration in the previous section:

In [61]:
prediction_df = pd.read_csv(f'{data_folder}/{experiment_name}/results_tutorial5_train_epoch_4.csv', sep=';')
prediction_df

,section_id,file_name,xmin,ymin,xmax,ymax,detection_score
0,102_A.jpg@3297-4216-2777-3165,102_A.jpg,3297,2777,4216,3165,0.978597
1,102_A.jpg@3221-3544-5465-6349,102_A.jpg,3221,5465,3544,6349,0.967986
2,102_A.jpg@2719-3498-4852-5194,102_A.jpg,2719,4852,3498,5194,0.961160
3,102_A.jpg@3528-3867-4057-5085,102_A.jpg,3528,4057,3867,5085,0.954487
4,102_A.jpg@2171-2839-1868-2386,102_A.jpg,2171,1868,2839,2386,0.941419
...,...,...,...,...,...,...,...
107831,9_D.jpg@3111-3692-2444-3082,9_D.jpg,3111,2444,3692,3082,0.051561
107832,9_D.jpg@2825-3153-1301-1698,9_D.jpg,2825,1301,3153,1698,0.051457
107833,9_D.jpg@2110-2584-1101-1989,9_D.jpg,2110,1101,2584,1989,0.051006
107834,9_D.jpg@903-1636-3752-4494,9_D.jpg,903,3752,1636,4494,0.050689


The statistics show us that, in fact, the model has quite low confidence values (<= 0.35) for 50% of the detections:

In [62]:
prediction_df.describe()

,xmin,ymin,xmax,ymax,detection_score
count,107836.000000,107836.000000,107836.000000,107836.000000,107836.000000
mean,3007.071822,3575.800271,3449.560601,4103.073899,0.426875
std,1443.705168,1609.879749,1475.469295,1637.756149,0.327494
min,0.000000,0.000000,243.000000,248.000000,0.050003
25%,1888.000000,2285.000000,2310.000000,2801.000000,0.100776
50%,2837.500000,3474.000000,3295.000000,4042.000000,0.344173
75%,3953.000000,4792.000000,4420.000000,5373.000000,0.765656
max,7862.000000,7965.000000,8141.000000,8191.000000,0.993109


To investigate how this impacts prediction quality, we first run the evaluation function we use to calculate leaderboard scores against these predictions for different IoU thresholds.\
The reported score is the sum of the _detection_acc@X_ values.

In [63]:
ground_truth = load_sections_df(f'{data_folder}/gdsc_train.csv')

evaluator = PredictionEvaluator(ground_truth)
thresholds = [0.5, 0.6, 0.7]
get_leaderboard_score(prediction_df, thresholds, evaluator)

2022-06-07 17:34:34,208 - gdsc_score - INFO - Computing results for threshold: 0.5
2022-06-07 17:34:34,209 - PredictionEvaluator - INFO - Matching sections


100%|██████████| 994/994 [01:24<00:00, 11.77it/s]

2022-06-07 17:35:58,707 - PredictionEvaluator - INFO - Merging matched sections


2022-06-07 17:35:58,850 - PredictionEvaluator - INFO - Done matching sections
2022-06-07 17:35:58,854 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-07 17:35:58,855 - PredictionEvaluator - INFO - Computing overall scores
2022-06-07 17:35:58,873 - gdsc_score - INFO - Computing results for threshold: 0.6
2022-06-07 17:35:58,874 - PredictionEvaluator - INFO - Matching sections


100%|██████████| 994/994 [01:25<00:00, 11.67it/s]

...

2022-06-07 17:37:24,222 - PredictionEvaluator - INFO - Done matching sections
2022-06-07 17:37:24,230 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-07 17:37:24,234 - PredictionEvaluator - INFO - Computing overall scores
2022-06-07 17:37:24,255 - gdsc_score - INFO - Computing results for threshold: 0.7
2022-06-07 17:37:24,256 - PredictionEvaluator - INFO - Matching sections


100%|██████████| 994/994 [01:25<00:00, 11.61it/s]

2022-06-07 17:38:49,932 - PredictionEvaluator - INFO - Merging matched sections


2022-06-07 17:38:50,073 - PredictionEvaluator - INFO - Done matching sections
2022-06-07 17:38:50,081 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-07 17:38:50,082 - PredictionEvaluator - INFO - Computing overall scores


{'detection_acc@iou0.5': 45.3,
 'detection_tp@iou0.5': 54153,
 'detection_fp@iou0.5': 53855,
 'detection_fn@iou0.5': 11534,
 'detection_acc@iou0.6': 41.85,
 'detection_tp@iou0.6': 51219,
 'detection_fp@iou0.6': 56703,
 'detection_fn@iou0.6': 14468,
 'detection_acc@iou0.7': 34.89,
 'detection_tp@iou0.7': 44896,
 'detection_fp@iou0.7': 62977,
 'detection_fn@iou0.7': 20791,
 'score': 122.04}

Now we run the function again &mdash; but this time only on detected regions with a confidence score greater than 0.5:

In [64]:
get_leaderboard_score(prediction_df[prediction_df.detection_score>0.5], thresholds, evaluator)

2022-06-07 17:38:50,126 - gdsc_score - INFO - Computing results for threshold: 0.5
2022-06-07 17:38:50,127 - PredictionEvaluator - INFO - Matching sections


100%|██████████| 994/994 [01:14<00:00, 13.40it/s]

2022-06-07 17:40:04,391 - PredictionEvaluator - INFO - Merging matched sections


2022-06-07 17:40:04,515 - PredictionEvaluator - INFO - Done matching sections
2022-06-07 17:40:04,518 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-07 17:40:04,519 - PredictionEvaluator - INFO - Computing overall scores
2022-06-07 17:40:04,534 - gdsc_score - INFO - Computing results for threshold: 0.6
2022-06-07 17:40:04,536 - PredictionEvaluator - INFO - Matching sections


100%|██████████| 994/994 [01:14<00:00, 13.41it/s]

2022-06-07 17:41:18,720 - PredictionEvaluator - INFO - Merging matched sections
2022-06-07 17:41:18,833 - PredictionEvaluator - INFO - Done matching sections
2022-06-07 17:41:18,840 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-07 17:41:18,841 - PredictionEvaluator - INFO - Computing overall scores
2022-06-07 17:41:18,853 - gdsc_score - INFO - Computing results for threshold: 0.7
2022-06-07 17:41:18,854 - PredictionEvaluator - INFO - Matching sections



100%|██████████| 994/994 [01:13<00:00, 13.44it/s]

2022-06-07 17:42:32,870 - PredictionEvaluator - INFO - Merging matched sections


2022-06-07 17:42:32,991 - PredictionEvaluator - INFO - Done matching sections
2022-06-07 17:42:32,998 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-07 17:42:33,003 - PredictionEvaluator - INFO - Computing overall scores


{'detection_acc@iou0.5': 58.97,
 'detection_tp@iou0.5': 41128,
 'detection_fp@iou0.5': 4055,
 'detection_fn@iou0.5': 24559,
 'detection_acc@iou0.6': 56.47,
 'detection_tp@iou0.6': 39982,
 'detection_fp@iou0.6': 5120,
 'detection_fn@iou0.6': 25705,
 'detection_acc@iou0.7': 50.0,
 'detection_tp@iou0.7': 36920,
 'detection_fp@iou0.7': 8154,
 'detection_fn@iou0.7': 28767,
 'score': 165.44}

We can see that, when relying on a minimal detection score, the number of false positives decreases by about an order of magnitude for all IoU thresholds.\
Consequently, we achieve a significantly higher accuracy.
Also, precision improves since the ratio of true positives to false positives increased.

On the other hand, the raising number of false negatives reflects that our model far more often fails to detect true sections: It doubles for IoU thresholds 0.5 and 0.6. It would be worthwhile to examine the detection score for these sections to learn how certain our model is about them.

Next, we test the influence on the leaderboard score by exporting and submitting filtered test predictions.

In [65]:
prediction_df = pd.read_csv(f'{data_folder}/{experiment_name}/results_tutorial5_test_epoch_4.csv', sep=';')
restricted_prediction_df = prediction_df[prediction_df.detection_score>0.5]
restricted_prediction_df.to_csv(f'{data_folder}/results_tutorial5_epoch_4_min_detection_score.csv', sep=';')

Great news, our assumption was correct! With this measure, we can boost our model's score to 90.

**Exercises:**
- Pick some nodules from the training data and check which detected sections are below the detection score threshold. Do you agree to sort out these detections?
- Use the model of Tutorial 4 together with a detection score threshold of 0.5 to generate predictions on training data. How does the calculated leaderboard score compare to the results we obtained here with ```cfg.model.test_cfg.rcnn.max_per_img = 400```?
- Find a threshold that further improves your model's performance. How can you find an adequate threshold with a low number of training runs? A good method saves time, energy, and money. Do not use a brute-force search!

## Do we Overfit?

Having conducted some experiments with the setup of Tutorial 4, we see that the mAP metric stagnates and sometimes starts fluctuating around ~0.65 after 2-3 epochs:
<br>
<br>
<img src="figures/overfitting_logs.png" alt="Do we overfit?" style="width:40%;" />
    
In addition, we observe that we obtain much higher leaderboard scores on training than on test data. For instance, 165 vs. 90 for the experiment in the previous section.  

Hypothesis:
> Overfitting impairs our model's generalizability.

To test this hypothesis, it is important to understand what _overfitting_ is and how it relates to _underfitting_.

### Overfitting

_Overfitting_ describes a situation in which a model is able to remember too many little details of the training data.
In the worst case, the model simply memorizes the input-output relation for every training example.\
While an overfitting model performs very well on training data, it underperforms on validation/test data.
In other words, it lacks ability to _generalize_.

Overfitting can be identified by comparing training to validation/test losses:
- Overfitting is indicated if the validation/test loss stagnates while the training loss continuous to improve.
- If, in further epochs, the validation/test loss starts getting worse again, the model clearly overfits.

<div style="text-align: left">
    <img src="figures/overfitting_loss.png" alt="Overfitting - Loss" style="width:550px;" />
</div>

(Figure taken from [IBM Cloud Learn Hub: Overfitting](https://www.ibm.com/cloud/learn/overfitting))

### Underfitting

_Underfitting_ occurs if the model is not able to capture relevant patterns from the training data.
As a result, the model makes a lot of wrong predictions so that its training and validation/test losses remain at a high level.\
There are several reasons for underfitting. For example:
- Lack of model complexity
- Use of unsuitable or insufficient features
- Insufficient number of training iterations

### Overfitting vs. Underfitting

The following figure compares an underfitting (left), a well generalizing (middle), and an overfitting model (right):

<div style="text-align: left">
    <img src="figures/underfitting_overfitting.png" alt="Overfitting" style="width:75%" />
</div>

(Figure taken from [scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html))


**Further reading:**
- [This article](https://towardsdatascience.com/overfitting-vs-underfitting-a-complete-example-d05dd7e19765) outlines over-/underfitting in more detail.
- [This article](https://towardsdatascience.com/8-simple-techniques-to-prevent-overfitting-4d443da2ef7d) presents different techniques you can apply to deal with overfitting. 
- [This article](http://scott.fortmann-roe.com/docs/BiasVariance.html) provides a detailed discussion on the [bias-variance tradeoff](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff) and its relation to over- and underfitting.


### Do we Really Overfit?

Based on the definitions above, we cannot come to the conclusion that our model overfits.\
The main reason for this is that our training logs do not contain information on how the validation/test loss develops in comparison to the training loss. 

To check if we overfit, we train a new model with our most recent configuration for 8 epochs. Afterwards, we compute the leaderboard score and compare it with the one we obtained for 4 epochs.

In [57]:
entry_point = 'tutorial_5_training_8_epochs.py'
exp_name = entry_point.split('.')[0].replace('_', '-')  # AWS does not allow . and _ as experiment names

account_id = boto3.client('sts').get_caller_identity().get('Account')
role = get_execution_role()

sm_client = boto3.client("sagemaker", region_name=DEFAULT_REGION)
sess = Session(sagemaker_client=sm_client)
s3_output_location = f"s3://{sess.default_bucket()}/{exp_name}"
input_channels = {"train": f"s3://{DEFAULT_BUCKET}"}

data_folder = str(PROJECT_DIR / 'data')
cfg, base_file = load_config_5_with_8_epochs(data_folder)

hyperparameters = extract_hyperparams(entry_point) # custom function to parse the training script and extract config
hyperparameters['base_file'] = base_file

metrics = [
    {"Name": "train:loss_rpn_cls", "Regex": "loss_rpn_cls: ([0-9\.]+)"},
    {"Name": "train:loss_rpn_bbox", "Regex": "loss_rpn_bbox: ([0-9\.]+)"},
    {"Name": "train:loss_cls", "Regex": "loss_cls: ([0-9\.]+)"},
    {"Name": "train:loss_bbox", "Regex": "loss_bbox: ([0-9\.]+)"},
    {"Name": "train:loss", "Regex": "loss: ([0-9\.]+)"},
    {"Name": "train:accuracy", "Regex": "acc: ([0-9\.]+)"},
    {"Name": "train:epoch", "Regex": "Epoch (\[[0-9\.]+\])"},
    {"Name": "val:epoch", "Regex": "Epoch\(val\) (\[[0-9]+\])"},
    {"Name": "val:mAP", "Regex": "mAP: ([0-9\.]+)"},
]

estimator = PyTorch(
    entry_point=entry_point,             # This function will be called by the training job
    source_dir="../src",                 # All code in this folder will be copied over
    image_uri=f"954362353459.dkr.ecr.{DEFAULT_REGION}.amazonaws.com/sm-training-custom:torch-1.8.1-cu111-noGPL",
    role=role,
    output_path=s3_output_location,
    container_log_level=20,             # 10=debug, 20=info
    base_job_name=exp_name,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",     # a GPU instance
    volume_size=45,
    metric_definitions=metrics,
    hyperparameters=hyperparameters,
)

estimator.fit(
    input_channels,
    wait=False,           # Whether or not the notebook should wait for the job to finish. By setting it to False we can continue working while the job runs on another machine.
)

# save the name of the experiment to the filesystem so that we can use it later
experiment_name = estimator._hyperparameters["sagemaker_job_name"]

with open(f'{PROJECT_DIR}/experiment_tut5_epoch_8.txt', 'w+') as f:
    f.write(experiment_name)

2022-06-07 17:28:07,449 - sagemaker.image_uris - INFO - Defaulting to the only supported framework/algorithm version: latest.
2022-06-07 17:28:07,470 - sagemaker.image_uris - INFO - Ignoring unnecessary instance type: None.
2022-06-07 17:28:07,914 - sagemaker - INFO - Creating training-job with name: tutorial-5-training-8-epochs-2022-06-07-17-28-07-449


In [7]:
# read the experiment name from the filesystem
with open(f'{PROJECT_DIR}/experiment_tut5_epoch_8.txt', 'r') as f:
    experiment_name = f.read()

model_location = f'{s3_output_location}/{experiment_name}/output/model.tar.gz'
local_model_dir = download_and_extract_model(model_uri=model_location, local_dir='data')

FileNotFoundError: [Errno 2] No such file or directory: '/home/sagemaker-user/gdsc5-tutorials-public/experiment_tut5_epoch_8.txt'

In [6]:
prediction_df = pd.read_csv(f'{data_folder}/{experiment_name}/results_tutorial5_test_epoch_8.csv', sep=';')
restricted_prediction_df = prediction_df[prediction_df.detection_score>0.5]
restricted_prediction_df.to_csv(f'{data_folder}/results_tutorial5_epoch_8_min_detection_score.csv', sep=';')

NameError: name 'experiment_name' is not defined

Training the model for 8 epochs leads to a significantly higher leaderbord score of 110, instead of 90 for 4 epochs. So the model is clearly not overfitting!\
However, increasing model performance by training for more and more epochs is not an option. This is simply too time-consuming and too costly.

Maybe our model just shows a low _convergence rate_ because it is not able to extract all relevant features from the input data?\
We will cover this in the following exercises.

**Exercises:**
- Think about possibilities to support our model in extracting features from input data. Hint: Check the size of the images we loaded in Tutorial 2, and compare them to the scale of the images in the model's configuration (look out for the parameter "img_scale").
- Evaluate how the model performs on different stainings or studies. Should we train individual models for them?

## Summary

This tutorial showed how we can systematically improve our model in an interative manner:
- We learned how to apply CRISP-DM and the scientific method to plan, prioritize, and carry out iterations.
- We formulated an initial list of hypotheses and adapted them on the basis of gained insights.
- We made our model more powerful by increasing the maximal number of detectable sections.
- We improved the model's predictions by making use of the provided detection scores as a measure of confidence.
- We discussed properties of over- and underfitting and derived further hypotheses.

Now you should be well-equipped to tackle the Global Data Science Challenge 2022.\
We wish you and your team good luck and a lot of fun!